# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.235104628024                   -0.50    9.0         
  2   -7.250458031463       -1.81       -1.40    1.0   5.13ms
  3   -7.251195468465       -3.13       -2.18    1.0   5.06ms
  4   -7.251192058559   +   -5.47       -2.07    2.0   6.26ms
  5   -7.251336657333       -3.84       -2.76    1.0   5.15ms
  6   -7.251338549983       -5.72       -3.42    1.0   5.15ms
  7   -7.251338712119       -6.79       -3.56    2.0   6.34ms
  8   -7.251338796153       -7.08       -4.26    1.0   5.37ms
  9   -7.251338798349       -8.66       -5.03    1.0   5.45ms
 10   -7.251338798692       -9.46       -5.29    2.0   6.68ms
 11   -7.251338798703      -10.97       -5.80    1.0   5.63ms
 12   -7.251338798704      -11.90       -6.02    2.0   6.75ms
 13   -7.251338798704      -12.42       -6.50    1.0   5.75ms
 14   -7.251338798705      -13.18       -6.98    3.0   7.26ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.055312219919247715
[ Info: Arnoldi iteration step 2: normres = 0.6220609624568683
[ Info: Arnoldi iteration step 3: normres = 0.6477624268987427
[ Info: Arnoldi iteration step 4: normres = 0.3163735132009867
[ Info: Arnoldi iteration step 5: normres = 0.3162829214345903
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.05e-02, 6.86e-02, 2.47e-01, 1.82e-01, 3.01e-02)
[ Info: Arnoldi iteration step 6: normres = 0.4660566909140053
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (3.89e-03, 1.05e-01, 4.29e-01, 1.28e-01, 7.43e-02)
[ Info: Arnoldi iteration step 7: normres = 0.09439006106637726
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.96e-04, 1.74e-02, 3.35e-02, 4.22e-02, 6.48e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11031257454416346
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (9.51e-